In [82]:
from transformers import GPTNeoForCausalLM, PreTrainedTokenizerFast

In [129]:
tokenizer_file = "./tokenizers/tinystories.json"
model_dir = "./models/8ea118ddbe1141d3aefd963421205232/9000"

In [130]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_file)
model = GPTNeoForCausalLM.from_pretrained(model_dir, local_files_only=True)

In [121]:
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(trainable_parameters)

total_parameters = sum(p.numel() for p in model.parameters())
print(total_parameters)

1104000
1104000


In [5]:
model.generation_config

GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "transformers_version": "4.32.1"
}

In [126]:
text_input = "Once upon a time,"

In [131]:
token_input = tokenizer(text_input, return_tensors="pt")
token_output = model.generate(**token_input, num_beams=1, max_length=80, temperature=1.0)
text_output = tokenizer.batch_decode(token_output, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [132]:
text_output

[' Once upon a time,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,']

In [ ]:
for mn, m in model.named_modules():
    for pn, p in m.named_parameters():
        fpn = f"{mn}.{pn}" if mn else pn
        print(fpn)